Скрипт собирает данные финансовой отчетности предприятий с сайта налоговой https://bo.nalog.ru/

Скрипт выгружает только общую информацию и финансовый отчет за 3 года.

Часть первая, собирает урлы.

Дата последних правок: 2021-12-13

## Библиотеки и настройки ##

In [ ]:
#Загружаю нужные библиотеки

import pandas as pd

# для юзер-агента
from fake_useragent import UserAgent

# прогресс-бар
from tqdm.notebook import trange, tqdm

# для пауз при парсинге
from time import sleep

# селениум
from selenium import webdriver

from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [ ]:
# настройки юзер-агента

ua = UserAgent()

# юзер-агент для Хрома
user_agent = ua.chrome

In [ ]:
test_url = 'https://intoli.com/blog/not-possible-to-block-chrome-headless/chrome-headless-test.html'

In [ ]:
# путь к файлам

file_path = 'C:/00_Data/ria/'
proxy_list_path = 'C:/work/proxy_list.txt'

In [ ]:
# загружаю список прокси
# список простой текстовый файл, который содержит прокси в формате: адрес:порт логин:пароль

proxy_list = []

with open(proxy_list_path, 'r') as f:
    for line in f:
        proxy_list += [line.split()]

In [ ]:
print('Рабочих прокси — ' + str(len(proxy_list)))

In [ ]:
# номер прокси проще прописать вручную, ввод со строки постоянно глючит

print('Номер прокси, от 0 до ' + str(len(proxy_list) - 1))
proxy_number = int(input())

#proxy_number = 1

In [ ]:
# настройки веб-драйвера, чтобы скрыть автоматическое управление

opt = webdriver.ChromeOptions()
opt.add_argument(f'user-agent={user_agent}')
opt.add_argument('--disable-blink-features')
opt.add_argument('--disable-blink-features=AutomationControlled')
#opt.add_argument(f'--proxy-server={proxy_list[proxy_number][0]}') # "девственный" прокси(:
opt.add_experimental_option("excludeSwitches", ["enable-automation"]) # отключает строку про удаленное управление
opt.add_experimental_option('useAutomationExtension', False)

In [ ]:
print('Login: ' + proxy_list[proxy_number][1])
print('Pass: ' + proxy_list[proxy_number][2])

In [ ]:
# запускаю драйвер, проверяю настройки

driver = webdriver.Chrome(options=opt)
driver.get(test_url)
driver.maximize_window() # раскрываю окно на максимум

### Функции ###

Чтобы сделать код менее громоздким, повторяющиеся куски вынес в функции.

In [ ]:
# функция закрывает окно-предупреждение

def close_alert():
    
    try: # ищу предупреждение и нажимаю на крестик
        driver.find_element_by_class_name('short-info-bottom').find_element_by_tag_name('svg').click()
        
    except:
        pass  

### 1 этап. Собираю ссылки по ИНН ###

In [ ]:
# ссылка

url_part_1 = 'https://bo.nalog.ru/search?query='

In [ ]:
# загружаю файл со списком ИНН

work_links = pd.read_excel(file_path + 'lic_gun_trade.xlsx', sheet_name='trade_2021_12_28')

In [ ]:
work_links.head(2)

In [ ]:
work_links.shape

In [ ]:
# список ИНН

inn_list = list(work_links['inn'].unique())

In [ ]:
len(inn_list)

### Код парсера ###

**Важно** ИНН я храню в формате "ИНН: 0666..", чтобы не возиться с форматами и "пропадающими" нулями. Если хранить ИНН как последовательность цифр, при выгрузке нули в начале неизбежно исчезнут, так как по умолчанию Питон решит, что это числа (int или float).

In [ ]:
# собираю ссылки на страницы

data = [] # список для данных
not_need = 'ПО ЗАДАННЫМ КРИТЕРИЯМ ПОИСКА СВЕДЕНИЙ НЕ НАЙДЕНО.'

In [ ]:
# код парсера

for inn in tqdm(inn_list[800:]): # цикл для перебора ИНН

    driver.get(url_part_1 + str(inn[5:]))
    sleep(2)
    driver.implicitly_wait(5) # возможно, стоит поставить явную задержку с большим таймингом
    
    # закрываю окно-предупреждение, если оно есть
    close_alert()
    
    try: # проверяю, есть ли данные
        
        # получаю текст заголовка h2
        h2_title = driver.find_element_by_class_name('content-text').find_element_by_tag_name('h2').text
        
        if h2_title == not_need: # если текст соответсвует стоп-фразе
            data.append([inn, 'not', 'none']) # добавляю none
            
        else:
            break
            
    except:
        url = driver.find_element_by_tag_name('a[data-query="' + str(inn[5:]) + '"]').get_attribute('href')
        
        data.append([inn, 'yes', url])

In [ ]:
# Записываю в датафрейм

head_list = ['inn', 'check', 'inn_page']

work_inn = pd.DataFrame(data, columns=head_list)

In [ ]:
work_inn.head(3)

In [ ]:
work_inn.shape

In [ ]:
# записываю в файл

work_inn.to_excel(file_path + 'nalog_url_1.xlsx', index=False)

In [ ]:
driver.quit()